# Classification of Nigerian Traditional Attire
### Step 1: Collection and Organization of the dataset

### Step 2: Spliting the dataset into training and validation set

In [1]:
import os
import random
import shutil

def split_dataset(source_dir, dest_dir, split_ratio=0.8):
    classes = os.listdir(source_dir)
    
    for cls in classes:
        cls_path = os.path.join(source_dir, cls)
        if not os.path.isdir(cls_path):
            continue

        images = [img for img in os.listdir(cls_path) if img.endswith(('.jpg', '.jpeg', '.png'))]
        random.shuffle(images)

        split_point = int(len(images) * split_ratio)
        train_imgs = images[:split_point]
        val_imgs = images[split_point:]

        for phase in ['train', 'val']:
            dest_cls_dir = os.path.join(dest_dir, phase, cls)
            os.makedirs(dest_cls_dir, exist_ok=True)

        # Copy images
        for img in train_imgs:
            shutil.copy(os.path.join(cls_path, img), os.path.join(dest_dir, 'train', cls, img))
        for img in val_imgs:
            shutil.copy(os.path.join(cls_path, img), os.path.join(dest_dir, 'val', cls, img))

    print("✅ Dataset split completed!")

# Usage
split_dataset('nigerian_attire', 'data_split', split_ratio=0.8)


✅ Dataset split completed!


### Step 3: Defining Augmentation and Transforms

In [2]:
from torchvision import transforms

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomAffine(degrees=15, translate=(0.05, 0.05)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


### Step 4: Loading Data With PyTorch Dataloaders


In [3]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

train_data = ImageFolder('data_split/train', transform=train_transforms)
val_data = ImageFolder('data_split/val', transform=val_transforms)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(val_data, batch_size=16, shuffle=False)
